# Experiment for the factorization of $\langle \nabla^k e_n, e_{(1^n)} \rangle$

In [6]:
# Parameters for the experiments
nn = 4
kk = 2

mu = [1]*nn

## Computation of the scalar product

In [7]:
# Setup of Ring of Symmetric functions
RR = FractionField(QQ['q','t'])
q,t = RR.gens()
Sym = SymmetricFunctions(RR)
e = Sym.e()
h = Sym.h()

# Compute the scalar product
left = e([nn])
left = left.nabla(power=kk)
right = e(mu)
poly_Hall = left.scalar(right)

## Computation of the factorization

First we need several functions

In [8]:
### First generate the set of all possible pmaj contributes
def gen_pmaj_contributes(nn,kk):
    # Generate all allowed images for pmaj contributes (multisets!)
    list_functions = [[0]]          # The lowest one must always be a 0
    for j in range(nn - 1):         # Add the new element to the set
        new_list = []
        for partial in list_functions:
            new_list = new_list + [ partial + [partial[j] + i] for i in range(kk+1)]
        list_functions = new_list
    #print(list_functions)

    # Compute all possible set partitions
    set_partitions = {}
    for lamb in Partitions(nn):                              # type: ignore
        temp = []
        for possible in OrderedSetPartitions(nn,lamb):       # type: ignore
            poss = [list(part) for part in possible]
            temp.append(poss)
        lambt = tuple(lamb)
        set_partitions.update({lambt:temp})

    # Now associate possible labels to these images
    list_pmaj_contributes = []
    for image in list_functions:                                                        # For every image of a pmaj contribute...
        partit_img = list(set(image))
        contr_multep = {i:sum([1 for v in image if v == i]) for i in range(nn*kk)}      # Dictionary storing for every contribute in the image its multeplicity
        partit_val = list(contr_multep.values())                                        # Define the list of these multeplicities
        
        read_order_incr = sorted(range(nn*kk), key=lambda x:-partit_val[x])             # The order in which we give read multeplicities
        
        partit_mult = tuple([v for v in sorted(partit_val,reverse=True) if v != 0])
        for choice in set_partitions[partit_mult]:
            max_interval = [i for i in range(len(partit_img)-1) if partit_img[i+1] - partit_img[i] == kk]         # Compute which contributes are exactly k distant
            new_funct = {}
            new_list = []
            for i in range(len(choice)):
                new_funct = new_funct | {val:read_order_incr[i] for val in choice[i]}
            
            check = [ max([v for v in range(1,nn+1) if new_funct[v] == partit_img[i+1]]) < min([v for v in range(1,nn+1) if new_funct[v] == partit_img[i]]) for i in max_interval ]
            if sum(check) == 0:
                new_list = [new_funct[j+1] for j in range(nn)]
                list_pmaj_contributes = list_pmaj_contributes + [new_list]
    return list_pmaj_contributes

#
### Define the pmaj_word from contribute
def pmaj_word(nn, kk, contribute, runlist = True):
    wpmaj = []
    runs = [[0]]
    bound = max(contribute)
    for val in range(bound + kk):            # Iterate for every run
        # Get the labels of this run
        lab_run = sorted([ v+1 for v in range(nn) if val < contribute[v] + kk and contribute[v] <= val],reverse=True)
        wpmaj = wpmaj + lab_run
        runs = runs + [lab_run]
    if runlist:
        return [wpmaj, runs]
    else:
        return wpmaj

#
### Max possible dinv
def maxdinv(nn, kk, contribute):
    maxim = []

    # Compute the pmaj word from the pmaj_contribute and order
    wpmaj = pmaj_word(nn,kk,contribute,runlist=False)
    spmaj = [wpmaj[i] for i in range(nn*kk) if wpmaj[i] not in wpmaj[:i]]

    return [kk*(spmaj.index(i)) for i in range(1,nn+1)]

#
### Define the list of values for beta for a given pmaj_word
def beta(nn, kk, contribute):
    betas_list = []

    # Compute the pmaj word from the pmaj_contribute
    [wpmaj, runs] = pmaj_word(nn,kk,contribute)

    for ind in range(1,nn+1):        # For every label find beta(label)
        h = min([hh for hh in range(len(runs)) if ind in runs[hh]])         # Find run in which ind appears first time
        beta_ind = sum([len(runs[j]) for j in range(h)]) + len([v for v in runs[h] if ind < v])
        betas_list = betas_list + [beta_ind]

    return betas_list

#
### Define the list of values for alpha for a given pmaj_word
def alpha(nn, kk, contribute):
    alphas_list = []

    # Compute the pmaj word from the pmaj_contribute
    [wpmaj, runs] = pmaj_word(nn,kk,contribute)

    for ind in range(1,nn+1):        # For every label find beta(label)
        h = min([hh for hh in range(len(runs)) if ind in runs[hh]])         # Find run in which ind appears first time
        alpha_ind = sum([len(runs[j]) for j in range(h-1)]) + len([v for v in runs[h-1] if ind < v])
        alphas_list = alphas_list + [alpha_ind]

    return alphas_list

#
### Define the list of values for u_m for a given pmaj_word
def u_m(nn, kk, contribute):
    u_list = []

    # Compute the pmaj word from the pmaj_contribute
    [wpmaj, runs] = pmaj_word(nn,kk,contribute)

    for ind in range(1,nn+1):        # For every label find beta(label)
        h = min([hh for hh in range(len(runs)) if ind in runs[hh]])         # Find run in which ind appears first time
        u_ind = len([v for v in runs[h-1] if v <= ind]) + len([v for v in runs[h] if ind < v])
        u_list = u_list + [u_ind]

    return u_list

#
### Contributes of maj
def word_maj(nn,kk,word):
    if kk == 0:
        return {i:0 for i in range(1,nn+1)}
    contributes = {}
    ascends = [word[i] <= word[i+1] for i in range(nn*kk - 1)] + [False]
    
    for i in range(1,nn+1):
        contr_i = sum(ascends[0:word.index(i)])
        contributes = contributes | {i:contr_i}
    return contributes

Now we can compute the factorization of the polynomial

In [9]:
from sage.combinat.q_analogues import q_int

### Let us compute the polynomial

poly_fact = 0*q*t

for contribute in gen_pmaj_contributes(nn,kk):
    # Compute the wpmaj and the corresponding q_exp
    [wpmaj,runs]= pmaj_word(nn,kk,contribute)
    q_exp = sum(word_maj(nn,kk,wpmaj).values())

    # Compute the t_analogue
    t_analogue = 1
    maxim = maxdinv(nn,kk,contribute)
    betas = beta(nn,kk,contribute)
    alphas = alpha(nn,kk,contribute)
    u_ms = [betas[i] - alphas[i] for i in range(nn)]
    for lab in range(1,nn+1):
        t_factor = 0*t
        for jj in range(alphas[lab-1],betas[lab-1]):
            t_factor = t_factor + t ** (maxim[lab-1] - jj)
        t_analogue = t_analogue * t_factor
    
    # Sum new contribute
    poly_fact += (q ** q_exp) * t_analogue 

print("Polynomial obtained through the factorization: {}".format(poly_fact))
print("Polynomial obtained through the Hall product:  {}".format(poly_Hall))


Polynomial obtained through the factorization: q^12 + q^11*t + q^10*t^2 + q^9*t^3 + q^8*t^4 + q^7*t^5 + q^6*t^6 + q^5*t^7 + q^4*t^8 + q^3*t^9 + q^2*t^10 + q*t^11 + t^12 + 3*q^11 + 4*q^10*t + 4*q^9*t^2 + 4*q^8*t^3 + 4*q^7*t^4 + 4*q^6*t^5 + 4*q^5*t^6 + 4*q^4*t^7 + 4*q^3*t^8 + 4*q^2*t^9 + 4*q*t^10 + 3*t^11 + 5*q^10 + 9*q^9*t + 10*q^8*t^2 + 10*q^7*t^3 + 10*q^6*t^4 + 10*q^5*t^5 + 10*q^4*t^6 + 10*q^3*t^7 + 10*q^2*t^8 + 9*q*t^9 + 5*t^10 + 6*q^9 + 14*q^8*t + 18*q^7*t^2 + 19*q^6*t^3 + 19*q^5*t^4 + 19*q^4*t^5 + 19*q^3*t^6 + 18*q^2*t^7 + 14*q*t^8 + 6*t^9 + 5*q^8 + 16*q^7*t + 25*q^6*t^2 + 28*q^5*t^3 + 29*q^4*t^4 + 28*q^3*t^5 + 25*q^2*t^6 + 16*q*t^7 + 5*t^8 + 3*q^7 + 14*q^6*t + 25*q^5*t^2 + 30*q^4*t^3 + 30*q^3*t^4 + 25*q^2*t^5 + 14*q*t^6 + 3*t^7 + q^6 + 9*q^5*t + 17*q^4*t^2 + 20*q^3*t^3 + 17*q^2*t^4 + 9*q*t^5 + t^6 + 4*q^4*t + 7*q^3*t^2 + 7*q^2*t^3 + 4*q*t^4 + q^3*t + q^2*t^2 + q*t^3
Polynomial obtained through the Hall product:  q^12 + q^11*t + q^10*t^2 + q^9*t^3 + q^8*t^4 + q^7*t^5 + q^6*t^6 + q^